In [3]:
import cv2
import numpy as np
import open3d as o3d
import os

data_path = "100831_155323_MultiCamera0_subset_db"
left_images = sorted([os.path.join(data_path, f) for f in os.listdir(data_path) if "cam0" in f])
right_images = sorted([os.path.join(data_path, f) for f in os.listdir(data_path) if "cam1" in f])

fx = 1000  
fy = 1000  
cx = 640   
cy = 360   
baseline = 0.2  

stereo = cv2.StereoBM_create(numDisparities=16 * 5, blockSize=15)

point_clouds = []
camera_poses = []

for left_img_path, right_img_path in zip(left_images, right_images):
    img_left = cv2.imread(left_img_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_img_path, cv2.IMREAD_GRAYSCALE)

    disparity = stereo.compute(img_left, img_right).astype(np.float32) / 16.0

    depth = (fx * baseline) / (disparity + 1e-6)  

    h, w = depth.shape
    Q = np.array([[1, 0, 0, -cx],
                  [0, -1, 0, cy],
                  [0, 0, fx, 0],
                  [0, 0, 0, 1]])

    points_3d = cv2.reprojectImageTo3D(disparity, Q)
    colors = cv2.cvtColor(cv2.imread(left_img_path), cv2.COLOR_BGR2RGB).reshape(-1, 3)
    mask = disparity > 0
    points = points_3d[mask]
    colors = colors[mask.reshape(-1)]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)
    point_clouds.append(pcd)

    if not camera_poses:
        camera_poses.append(np.eye(4))
    else:
        prev_pcd = point_clouds[-2]
        reg = o3d.pipelines.registration.registration_icp(
            pcd, prev_pcd, max_correspondence_distance=0.05,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint()
        )
        last_pose = camera_poses[-1] @ reg.transformation
        camera_poses.append(last_pose)

for i, pcd in enumerate(point_clouds):
    o3d.io.write_point_cloud(f"point_cloud_{i}.ply", pcd)

camera_trajectory = o3d.geometry.LineSet()
trajectory_points = [pose[:3, 3] for pose in camera_poses]
lines = [[i, i + 1] for i in range(len(trajectory_points) - 1)]
camera_trajectory.points = o3d.utility.Vector3dVector(trajectory_points)
camera_trajectory.lines = o3d.utility.Vector2iVector(lines)

vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Camera Trajectory", width=800, height=600)
vis.add_geometry(camera_trajectory)
vis.run() 
vis.destroy_window()  

out = cv2.VideoWriter('depth_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 10, (w, h))
for left_img_path, right_img_path in zip(left_images, right_images):
    img_left = cv2.imread(left_img_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_img_path, cv2.IMREAD_GRAYSCALE)
    disparity = stereo.compute(img_left, img_right).astype(np.float32) / 16.0
    depth_color = cv2.applyColorMap(cv2.convertScaleAbs(disparity, alpha=255/disparity.max()), cv2.COLORMAP_JET)
    out.write(depth_color)
out.release()
